In [21]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [24]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later
import google.generativeai

In [25]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC1


In [26]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [27]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

# Three way convo

In [28]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [60]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

In [72]:
number_of_words = "three"
system_prompt_base_improv_game = f"ONLY RETURN {number_of_words} WORDS. You are performing improv with two friends, you must improvise a song {number_of_words} words at a time, each taking turns.  You must only contribute {number_of_words} words a turn!"
system_prompt_beginner = system_prompt_base_improv_game +  " You are a beginner at improv and occasionally get a word with not quite the right meaning but generally you're rhythmic just not perfectly all the time!"
system_prompt_cheeky_expert = system_prompt_base_improv_game +  " You are an expert at improv but a bit cheeky in that you always get the right rhythm, but you always cheekily change the song to go somewhere unexpected with \
your word choice!  The amount you change the song direction varies"
system_prompt_expert = system_prompt_base_improv_game +  " You are an expert at improv and used to dealing with others getting the rhyme/meaning slightly wrong and you can either bring the conversation back or embrace a mistake. \
One thing is for sure, you're an entertainer!"

def reset_messages():
    gpt_messages = ["The wheels on"]#["The wheels on"]
    claude_messages = [""]#["the bus go"]
    gemini_messages = [""]#["round and round"]
    return gpt_messages, claude_messages, gemini_messages


gpt_messages, claude_messages, gemini_messages = reset_messages()

def call_ai(messages, model='llama3.2'):
    result = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )
    output = result.choices[0].message.content
    print(output)
    return output

def call_assistant(system_prompt, assistant_no):
    '''
    system_prompt: str -> explain your assistants role in the conversation
    assistant_no: int [0,2] -> 
    '''
    messages = [
        {'role': 'system', 'content':system_prompt}
    ]
    
    a_role = 'assistant' if assistant_no == 0 else 'user'
    b_role = 'assistant' if assistant_no == 1 else 'user'
    c_role = 'assistant' if assistant_no == 2 else 'user'
    
    for a, b, c in zip(a_messages, b_messages, c_messages):
        if a != "": messages.append({'role': a_role, 'content':a})
        if b != "": messages.append({'role': b_role, 'content':b}) 
        if c != "": messages.append({'role': c_role, 'content':c})
    
    if assistant_no == 0:
        pass
    elif assistant_no == 1:
        messages.append({'role': 'user', 'content':a_messages[-1]})
    elif assistant_no == 2:
        messages.append({'role': 'user', 'content':a_messages[-1]})
        messages.append({'role': 'user', 'content':b_messages[-1]})
    
    return call_ai(messages)


def call_beginner():
    messages = [
        {'role': 'system', 'content':system_prompt_beginner}
    ]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        if gpt != "": messages.append({'role': 'assistant', 'content':gpt})
        if claude != "": messages.append({'role': 'user', 'content':claude}) 
        if gemini != "": messages.append({'role': 'user', 'content':gemini})
    
    return call_ai(messages)


def call_cheeky():
    messages = [
        {'role': 'system', 'content':system_prompt_cheeky_expert}
    ]
    for gpt, cla, gem in zip(gpt_messages, claude_messages, gemini_messages):
        if gpt != "": messages.append({'role': 'user', 'content':gpt})
        if cla != "": messages.append({'role': 'assistant', 'content':cla})
        if gem != "": messages.append({'role': 'user', 'content':gem})
    messages.append({'role': 'user', 'content':gpt_messages[-1]})
    
    return call_ai(messages)


def call_expert():
    messages = [
        {'role': 'system', 'content':system_prompt_expert}
    ]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        if gpt != "": messages.append({'role': 'user', 'content':gpt})
        if claude != "": messages.append({'role': 'user', 'content':claude})
        if gemini != "": messages.append({'role': 'assistant', 'content':gemini})
    messages.append({'role': 'user', 'content':gpt_messages[-1]})
    messages.append({'role': 'user', 'content':claude_messages[-1]})
    
    return call_ai(messages)



In [73]:
print(
    gpt_messages,
    claude_messages,
    gemini_messages
)

['The wheels on'] [''] ['']


In [74]:
gpt_messages.append(call_beginner())

In [75]:
claude_messages.append(call_cheeky())

the bus go round


In [76]:
gemini_messages.append(call_expert())

round and round


In [77]:
def my_improv_song():
    reset_messages()
    for i in range(3):
        print("1Beginner: ", end="")
        gpt_messages.append(call_beginner())
        print("2Cheeky expert: ", end="")
        claude_messages.append(call_cheeky())
        print("3Expert: ", end="")
        gemini_messages.append(call_expert())

my_improv_song()

1Beginner: and never stop
2Cheeky expert: till they hit
3Expert: the ground with
1Beginner: a loud CRASH
2Cheeky expert: that echoes loudly
3Expert: and startling everyone
1Beginner: on the sidewalk
2Cheeky expert: including a kitten
3Expert: who's quite startled


# Adding Gradio

In [8]:
import gradio as gr

def my_func(inp1, inp2):
    return inp1 + inp2


view = gr.Interface(
    fn=my_func,
    inputs=[
        gr.Textbox(label="Beginner ai improv singer: Start an improv song with this line:"),
        gr.Textbox(label="Cheeky expert ai improv singer: and this optional 2nd line"),
        gr.Textbox(label="Expert ai improv singer: and this optional 3rd line")
    ],
    outputs=[
        gr.Markdown(label="Song (it may not be musical):")
    ],
    allow_flagging='never'
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
